# Rent price prediction for apartments in Cluj Napoca
First of all I need to import all the modules that I'm going to use and the dataset.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('scraping_env/webscrapy/webscrapy/rent_price_results.csv', thousands='.',decimal=',')
df.head()

,currency,floor,neighborhood,new_building,partitioning,price,rooms,space
0,EUR / lună,Etaj 5/7,"Cluj-Napoca, zona Central",NaN,Semidecomandat,400.0,2 camere,58.5 mp utili
1,EUR / lună,Etaj 8/10,"Cluj-Napoca, zona Mănăştur",NaN,Semidecomandat,280.0,2 camere,52 mp utili
2,EUR / lună,Etaj 3/3,"Cluj-Napoca, zona Zorilor",Bloc nou,Semidecomandat,400.0,3 camere,84 mp utili
3,EUR / lună,Parter/4,"Cluj-Napoca, zona Gheorgheni",NaN,Semidecomandat,400.0,3 camere,65 mp utili
4,EUR / lună,Etaj 1/2,"Cluj-Napoca, zona Zorilor",Bloc nou,Decomandat,420.0,2 camere,55 mp utili


In [3]:
df.shape

(5005, 8)

## 1. Data Cleaning

In my first step I want to clean the data from my dataset. First of all I want to see all the null values in my dataset and decide how to manage them.

In [4]:
df.isnull().sum()

currency           2
floor              1
neighborhood       0
new_building    3460
partitioning      66
price              2
rooms              0
space              0
dtype: int64

Because for 'floor', 'currency' and 'price' there are a few null values, respectively 1 and 2 and I suspect that those from the 'currency' and 'price' are at the same index, I will drop them.

'New_building' has so many null values because on the site, from where I scraped the data and with the help of which I built my dataset, only the new buildings were tagged as 'new bulding' the other ones didn't have any labels. So I will consider them 'old buildings'.

For 'partitioning' I don't see for now any reason why there are 65 null values, but in my further analysis I will try to figure out how to manage them.

In [5]:
df.dropna(subset=['floor', 'price'], inplace=True)

In [6]:
df.isnull().sum()

currency           0
floor              0
neighborhood       0
new_building    3457
partitioning      65
price              0
rooms              0
space              0
dtype: int64

For 'currency' feature I want to see how may categories I have and how to clean it.

In [7]:
df.groupby('currency')['currency'].count()

currency
EUR + TVA / lună      48
EUR / lună          4946
RON / lună             8
Name: currency, dtype: int64

We can see that I have two categories that both refer to the curreny euro and one for ron. In this case I want to make two categories for this feature one for euro and one for ron. When I scraped the data I wanted to extract this feature because I know that there is a posability that some rent prices are listed in ron so when I do the analysis and built the model, if I don't trasnform the prices to be at the same currency then my data will be distorted. 

In [8]:
df['currency'] = df['currency'].apply(lambda x: str(x.split(' ')[0]))

In [9]:
df.groupby('currency')['currency'].count()

currency
EUR    4994
RON       8
Name: currency, dtype: int64

A problem that I noticed in my dataset was that in some rows my values were swaped between columns so I want to fix this issue. That's way my next step is to clean the feature 'partitioning' and first I want to see all the categories it has.

In [10]:
df.groupby('partitioning')['partitioning'].count()

partitioning
Bloc nou            15
Circular             9
Decomandat        2346
Imobil nou           1
Nedecomandat        50
Semidecomandat    2508
Vagon                8
Name: partitioning, dtype: int64

We can see that in the categories in 'partitioning' we also have 'Bloc nou' and 'Imobil nou'. I want to see the rows where these categories are to see if there are swapt rows betweeen this freature and other ones.

In [11]:
df['partitioning'][df['partitioning'] == 'Bloc nou'].index.tolist()

[263,
 706,
 878,
 917,
 1793,
 2061,
 2220,
 2475,
 2689,
 3674,
 3827,
 4238,
 4340,
 4750,
 4903]

In [12]:
print(df.loc[[263, 706, 878, 917]])

    currency     floor                  neighborhood new_building  \
263      EUR  Etaj 3/6      Cluj-Napoca, zona Europa          NaN   
706      EUR  Etaj 5/8  Cluj-Napoca, zona Gheorgheni          NaN   
878      EUR  Etaj 1/4   Cluj-Napoca, zona Buna Ziua          NaN   
917      EUR    Etaj 4   Cluj-Napoca, zona Bună Ziua          NaN   

    partitioning  price     rooms        space  
263     Bloc nou  700.0  2 camere  56 mp utili  
706     Bloc nou  465.0  2 camere  72 mp utili  
878     Bloc nou  600.0  3 camere  76 mp utili  
917     Bloc nou  900.0  3 camere  80 mp utili  


We can see that I have null values in my new_building column and in my partitioning column I have the new building category 'Bloc nou'. So in this case I think the best approach is to swap at this indexes the values between the two columns. I will do the same thing for the row where in 'partitioning' feature there is the new building category 'Imobil nou'. For resolving this problem I will make a function.

In [13]:
def switch_rows_partitioning(df, variable, element):
    for content in df[variable]:
        if element == content:
            df[[variable,'new_building']] = df[['new_building',variable]].where(df[variable] == element, df[[variable,'new_building']].values)

In [14]:
switch_rows_partitioning(df,'partitioning','Bloc nou')
switch_rows_partitioning(df,'partitioning','Imobil nou')

In [15]:
df.groupby('partitioning')['partitioning'].count()

partitioning
Circular             9
Decomandat        2346
Nedecomandat        50
Semidecomandat    2508
Vagon                8
Name: partitioning, dtype: int64

Next I want to see all the categories from the 'floor' feature.

In [16]:
df.groupby('floor')['floor'].count().head(-50)

floor
Decomandat      2
Demisol         5
Demisol/1      10
Demisol/2       8
Demisol/3       3
Demisol/4       5
Demisol/5       2
Etaj 1         66
Etaj 1/1      122
Etaj 1/10      62
Etaj 1/11      10
Etaj 1/2      127
Etaj 1/20       1
Etaj 1/3      151
Etaj 1/4      342
Etaj 1/5       55
Etaj 1/6       63
Etaj 1/7       23
Etaj 1/8       70
Etaj 1/9       10
Etaj 10         4
Etaj 10/10     49
Etaj 10/11     21
Etaj 10/12      1
Etaj 11/11      3
Etaj 11/12      3
Etaj 12/12      1
Etaj 2         60
Etaj 2/10      55
Etaj 2/11      11
Etaj 2/2      133
Etaj 2/3      136
Etaj 2/4      327
Etaj 2/5       37
Etaj 2/6       44
Etaj 2/7       23
Etaj 2/8       54
Etaj 2/9        9
Etaj 3         44
Etaj 3/10      54
Etaj 3/11      19
Etaj 3/12       1
Etaj 3/3       76
Etaj 3/4      335
Etaj 3/5       73
Etaj 3/6       61
Etaj 3/7       24
Etaj 3/8       55
Etaj 3/9        7
Etaj 4         24
Etaj 4/10      50
Etaj 4/11       3
Etaj 4/12       1
Etaj 4/4      237
Etaj 4/5       47
Etaj

We can see that, first of all, I have a category 'Decomandat' that is specific to the feature 'partitioning'. In this case I want to see if at the indexes of this category the rows are swaped between the columns.

In [17]:
df['floor'][df['floor'] == 'Decomandat'].index.tolist()

[212, 4903]

In [18]:
print(df.loc[[212, 4903]])

     currency       floor                  neighborhood new_building  \
212       EUR  Decomandat  Cluj-Napoca, zona Gheorgheni          NaN   
4903      EUR  Decomandat    Cluj-Napoca, zona Bulgaria     Bloc nou   

     partitioning  price        rooms     space  
212           NaN  400.0     3 camere  Etaj 1/4  
4903          NaN  499.0  75 mp utili  Etaj 3/4  


We can see that in this case we also have swaped values, the values for 'floor' are at 'space' and the values for partitioning are at floor. So we need to swap these elements and I will consider the null values from partitioning to be from the feature 'space'. For resolving this issue I will implement a function and I will verify if the function was implemented as expected.

In [19]:
def switch_rows_floor(df, variable, element):
    for content in df[variable]:
        if element in content:
            df[[variable,'space']] = df[['space',variable]].where(df[variable] == element, df[[variable,'space']].values)
            df[['space','partitioning']] = df[['partitioning','space']].where(df['space'] == element, df[['space','partitioning']].values)

In [20]:
switch_rows_floor(df,'floor','Decomandat')

In [21]:
print(df.loc[[212, 4903]])

     currency     floor                  neighborhood new_building  \
212       EUR  Etaj 1/4  Cluj-Napoca, zona Gheorgheni          NaN   
4903      EUR  Etaj 3/4    Cluj-Napoca, zona Bulgaria     Bloc nou   

     partitioning  price        rooms space  
212    Decomandat  400.0     3 camere   NaN  
4903   Decomandat  499.0  75 mp utili   NaN  


I managed to solve the issues above. Now I want to evaluate again the categories from 'floor' feature to see if I can find further issues.

In [22]:
df.groupby('floor')['floor'].count().head(-1)

floor
Demisol            5
Demisol/1         10
Demisol/2          8
Demisol/3          3
Demisol/4          5
                  ..
Parter/6          20
Parter/7          16
Parter/8          44
Parter/9           4
Semidecomandat     4
Name: floor, Length: 107, dtype: int64

We ca see that there are 4 rows with the category 'Semidecomandat' which is specific to the 'partitioning' feature. In this case I will do the same process as above. I will verify those rows and if their data is swaped I will implement the same function and verify if the changes were made properly. 

In [23]:
df['floor'][df['floor'] == 'Semidecomandat'].index.tolist()

[1468, 2689, 3827, 4340]

In [24]:
print(df.loc[[1468, 2689, 3827, 4340]])

     currency           floor                  neighborhood new_building  \
1468      EUR  Semidecomandat   Cluj-Napoca, zona Bună Ziua          NaN   
2689      EUR  Semidecomandat  Cluj-Napoca, zona Gheorgheni     Bloc nou   
3827      EUR  Semidecomandat    Cluj-Napoca, zona Mănăştur     Bloc nou   
4340      EUR  Semidecomandat     Cluj-Napoca, zona Zorilor     Bloc nou   

     partitioning  price     rooms       space  
1468          NaN  250.0  2 camere      Etaj 3  
2689          NaN  350.0  2 camere    Etaj 1/4  
3827          NaN  299.0  2 camere  Mansarda/4  
4340          NaN  300.0  2 camere    Etaj 1/4  


In [25]:
switch_rows_floor(df,'floor','Semidecomandat')

In [26]:
print(df.loc[[1468, 2689, 3827, 4340]])

     currency       floor                  neighborhood new_building  \
1468      EUR      Etaj 3   Cluj-Napoca, zona Bună Ziua          NaN   
2689      EUR    Etaj 1/4  Cluj-Napoca, zona Gheorgheni     Bloc nou   
3827      EUR  Mansarda/4    Cluj-Napoca, zona Mănăştur     Bloc nou   
4340      EUR    Etaj 1/4     Cluj-Napoca, zona Zorilor     Bloc nou   

        partitioning  price     rooms space  
1468  Semidecomandat  250.0  2 camere   NaN  
2689  Semidecomandat  350.0  2 camere   NaN  
3827  Semidecomandat  299.0  2 camere   NaN  
4340  Semidecomandat  300.0  2 camere   NaN  


In [27]:
df.groupby('floor')['floor'].count().head(10)

floor
Demisol        5
Demisol/1     10
Demisol/2      8
Demisol/3      3
Demisol/4      5
Demisol/5      2
Etaj 1        66
Etaj 1/1     122
Etaj 1/10     62
Etaj 1/11     10
Name: floor, dtype: int64

As we see above there are many categories in this column. My main aim here is to transform the categories to represent only the number of the floor the apartment is. For 'Demisol' I want to switch it with '0', that is the floor below first floor and for each floor the category to be represented only by the number of that specific floor.

In [28]:
df.loc[df['floor'].str.contains('Demisol'), 'floor'] = '0'

In [29]:
df.loc[df['floor'].str.contains('Etaj'), 'floor'] = df['floor'].str[5:]

In [30]:
df.groupby('floor')['floor'].count().head(-10)

floor
0              33
1              66
1/1           122
1/10           62
1/11           10
             ... 
Mansarda/4     10
Mansarda/5      4
Mansarda/7      1
Parter        116
Parter/1      122
Name: floor, Length: 92, dtype: int64

We can see that above I also have 'Mansarda' which is the top floor of a building and I will consider it as 100, I want it to have a higher number so that I now its the top floor. In Romania we don't have apartment building high as 100 floors so this abroach will be safe. I also have 'Parter' which is the first floor and I will associate it with '1'. 

In [31]:
df.loc[df['floor'].str.contains('Mansarda'), 'floor'] = '100'

In [32]:
df.loc[df['floor'].str.contains('Parter'), 'floor'] = '1'

In [33]:
df['floor'].value_counts().head(-1)

1                     928
1/4                   345
3/4                   336
2/4                   327
4/4                   237
                     ... 
Ultimele 2 etaje/4      2
12/12                   1
4/12                    1
3/12                    1
1/20                    1
Name: floor, Length: 84, dtype: int64

Now I want to pick up the value of the floor choosing the number until the character '/'. I also have the category 'Ultimele 2 etaje/4 ' and I will consider it as second floor.

In [34]:
df.loc[df['floor'].str.contains('Ultimele 2 etaje/4'), 'floor'] = '2'

In [35]:
df.loc[df['floor'].str.contains('/'), 'floor'] = df['floor'].str.split('/').str[0]

In [36]:
df.groupby('floor')['floor'].count()

floor
0        33
1      1967
10       75
100      54
11        6
12        1
2       891
3       751
4       470
5       228
6       150
7       158
8       118
9       100
Name: floor, dtype: int64

My next step is to clean up the data from the feature 'neighborhood'. I will start with verifying the different categories of this feature.  

In [37]:
df.groupby('neighborhood')['neighborhood'].count()

neighborhood
Cluj-Napoca, zona Aeroport                 4
Cluj-Napoca, zona Andrei Muresanu          4
Cluj-Napoca, zona Andrei Mureşanu        121
Cluj-Napoca, zona Aurel Vlaicu             7
Cluj-Napoca, zona Baciu                    2
Cluj-Napoca, zona Borhanci                47
Cluj-Napoca, zona Bulgaria                10
Cluj-Napoca, zona Buna Ziua               11
Cluj-Napoca, zona Bună Ziua              244
Cluj-Napoca, zona Calea Turzii            36
Cluj-Napoca, zona Central                765
Cluj-Napoca, zona Cordos                   1
Cluj-Napoca, zona Câmpului                 6
Cluj-Napoca, zona Dâmbul Rotund           17
Cluj-Napoca, zona Europa                 109
Cluj-Napoca, zona Exterior Vest            1
Cluj-Napoca, zona Gară                    26
Cluj-Napoca, zona Gheorgheni             855
Cluj-Napoca, zona Grigorescu             185
Cluj-Napoca, zona Gruia                   27
Cluj-Napoca, zona Haşdeu                  14
Cluj-Napoca, zona Horea                   

We can see that all the categories begin with 'Cluj-Napoca, zona ' and I can drop it. Other aspect that I observe is that some neighborhoods have duplicates and that is because their duplicates contain diacritics used in romanian language. Because of this I want to replace them with the name without those diacritics.

In [38]:
df['neighborhood'] = df['neighborhood'].str[18:]

In [39]:
df['neighborhood'].replace(to_replace=['Andrei Mureşanu', 'Bună Ziua', 'Câmpului', 'Dâmbul Rotund', 'Gară', 'Haşdeu', 'Între Lacuri', 'Mărăşti', 'Mănăştur', 'P-ţa Mihai Viteazul', 'Someşeni'],
           value= ['Andrei Muresanu', 'Buna Ziua', 'Campului', 'Dambul Rotund','Gara', 'Hasdeu', 'Intre Lacuri', 'Marasti', 'Manastur', 'P-ta Mihai Viteazul', 'Someseni'], 
           inplace=True)

In [40]:
df.groupby('neighborhood')['neighborhood'].count()

neighborhood
Aeroport                 4
Andrei Muresanu        125
Aurel Vlaicu             7
Baciu                    2
Borhanci                47
Bulgaria                10
Buna Ziua              255
Calea Turzii            36
Campului                 6
Central                765
Cordos                   1
Dambul Rotund           17
Europa                 109
Exterior Vest            1
Gara                    26
Gheorgheni             855
Grigorescu             185
Gruia                   27
Hasdeu                  14
Horea                   19
Intre Lacuri           155
Iris                    56
Manastur               546
Marasti                720
P-ta Mihai Viteazul     53
Plopilor               107
Semicentral            159
Someseni                15
Sopor                   26
Ultracentral            95
Zorilor                559
Name: neighborhood, dtype: int64

Next step I want to evalute the feature 'new_buiding' and I want to start with seeing it's categories.

In [41]:
df.groupby('new_building')['new_building'].count()

new_building
Bloc nou      1471
Imobil nou      90
Name: new_building, dtype: int64

The category 'Imobil nou' it's the same thing as 'Bloc nou', so I will replace 'Bloc nou' with 'Imobil nou', but first I need to manage the missing values. So as I said above, in this case, I want to replace the missing values with 'Imobil vechi', because, from where I scraped the data and with the help of which I built my dataset, only the new buildings were tagged as 'Imobil nou' the other ones didn't have any labels. So I will consider them 'Imobil vechi'.

In [42]:
df['new_building'].isnull().sum()

3441

In [43]:
df['new_building'] = df['new_building'].fillna('Imobil vechi')

The category 'Imobil nou' it's the same thing as 'Bloc nou', so I will replace it with 'Bloc nou'.

In [44]:
df.loc[df['new_building'].str.contains('Bloc nou'), 'new_building'] = 'Imobil nou'

In [45]:
df.groupby('new_building')['new_building'].count()

new_building
Imobil nou      1561
Imobil vechi    3441
Name: new_building, dtype: int64

For 'rooms' feature I want to start in the same manner by verifying its categories.

In [46]:
df['rooms'].value_counts()

2 camere       2771
3 camere       1222
o cameră        802
4 camere        185
5 camere         16
6 camere          2
50 camere         1
75 mp utili       1
7 camere          1
11 camere         1
Name: rooms, dtype: int64

We can see that I have a category with '75 mp utili' and I suspect it is another case of swaped column values in a row. I also want to extract only the number of rooms and get rid of the word 'camere'. I also noticed that I have the category 'o cameră' which means one room and I want to replace it with 1.

In [47]:
df['rooms'][df['rooms'] == '75 mp utili'].index.tolist()

[4903]

In [48]:
print(df.loc[[4903]])

     currency floor neighborhood new_building partitioning  price  \
4903      EUR     3     Bulgaria   Imobil nou   Decomandat  499.0   

            rooms space  
4903  75 mp utili   NaN  


In [49]:
df[['rooms','space']] = df[['space','rooms']].where(df['rooms'] == '75 mp utili', df[['rooms','space']].values)

In [50]:
df['rooms'] = df['rooms'].str[:-7]

In [51]:
df['rooms'].isnull().sum()

1

Because I swaped at the index 4903 the value of the columns I got a null value in my room feature so I want to fill it with the most frequent number of rooms in the dataset. 

In [52]:
df['rooms'] = df['rooms'].fillna(df['rooms'].mode()[0]) 

In [53]:
df['rooms'].isnull().sum()

0

In [54]:
df.loc[df['rooms'].str.contains('o'), 'rooms'] = '1'

In [55]:
df['rooms'].value_counts()

2     2772
3     1222
1      802
4      185
5       16
6        2
50       1
7        1
11       1
Name: rooms, dtype: int64

For the feature 'space' I want to see how the its values. Based on the data above the values seem to have measurement unit names because they present, along with the values, the measure of the space of each apartment but if they are the same unit of measurement I will keep only the values.

In [56]:
df.groupby('space')['space'].count().head(40)

space
1 mp utili         1
100 mp utili      66
101 mp utili       1
103 mp utili       3
104 mp utili       3
105 mp utili       9
106 mp utili       1
109 mp utili       2
11 mp utili        3
110 mp utili      20
117 mp utili       2
118 mp utili       3
119 mp utili       1
120 mp utili      19
125 mp utili       3
127 mp utili       1
130 mp utili       7
131 mp utili       2
134 mp utili       1
135 mp utili       2
136 mp utili       4
138 mp utili       1
14 mp utili        1
140 mp utili       6
145 mp utili       1
146 mp utili       1
150 mp utili       9
160 mp utili       2
162 mp utili       1
170 mp utili       2
175 mp utili       3
180 mp utili       4
189 mp utili       1
19 mp utili        1
20 mp utili        8
20.37 mp utili     1
200 mp utili       6
21 mp utili        8
210 mp utili       1
22 mp utili       25
Name: space, dtype: int64

In [57]:
df['space'] = df['space'].str[:-9]

After all the swaping I did above I want to see if I have null values in my 'space' feature.

In [58]:
df['space'].isnull().sum()

5

I have 5 null values and because there are not that many and as a result I want to fill them the median value of the whole feature. 

In [59]:
df['space'] = df['space'].fillna(df['space'].median()) 

In [60]:
df['space'].isnull().sum()

0

In this case I want the values of this feature to be floating number because they are continuous variables that represent the size of each apartment.

In [61]:
df['space'] = df['space'].astype(float)

'Price' feature represents our dependent variable, a continuous one, and I want to change its type to float. Also I want to identify the rows that the price of the rent is presented in the currency 'RON' and I want to change it in euros because, as we saw above, the majority of the prices are presented in euros and it will be more precise in this way. For this change I will be taking into account the value of 1 euro in romanian currency and I will consider it 4.875 ron/euro because the biggest values in the past three months was 4.8733 and in the present moment is 4.8715 so I want to take into account also a small error that can take place by the fluctuation of the exchange rate. This can be checked at this [exchange rate site](https://www.cursbnr.ro/).

In [62]:
df['price']=df['price'].astype(float)

In [63]:
df['currency'][df['currency'] == 'RON'].index.tolist()

[175, 177, 1952, 2213, 2694, 2985, 4176, 4854]

In [64]:
print(df.loc[[175, 177, 1952, 2213, 2694, 2985, 4176, 4854]])

     currency floor     neighborhood  new_building    partitioning   price  \
175       RON     1             Iris    Imobil nou      Decomandat  2400.0   
177       RON     2             Iris  Imobil vechi  Semidecomandat  1450.0   
1952      RON     0          Central  Imobil vechi      Decomandat  1350.0   
2213      RON     2          Marasti  Imobil vechi  Semidecomandat  1000.0   
2694      RON     3          Marasti  Imobil vechi  Semidecomandat  2000.0   
2985      RON     1         Manastur  Imobil vechi      Decomandat  1500.0   
4176      RON     4  Andrei Muresanu  Imobil vechi      Decomandat  1850.0   
4854      RON     1       Gheorgheni  Imobil vechi             NaN  1000.0   

     rooms  space  
175      2   55.0  
177      1   28.0  
1952     1   35.0  
2213     1   28.0  
2694     3   64.0  
2985     2   52.0  
4176     2   60.0  
4854     1   28.0  


In [65]:
df['price'] = (df['price']/float(4.875)).round().where(df['currency'] == 'RON', df['price'])

In [66]:
print(df.loc[[175, 177, 1952, 2213, 2694, 2985, 4176, 4854]])

     currency floor     neighborhood  new_building    partitioning  price  \
175       RON     1             Iris    Imobil nou      Decomandat  492.0   
177       RON     2             Iris  Imobil vechi  Semidecomandat  297.0   
1952      RON     0          Central  Imobil vechi      Decomandat  277.0   
2213      RON     2          Marasti  Imobil vechi  Semidecomandat  205.0   
2694      RON     3          Marasti  Imobil vechi  Semidecomandat  410.0   
2985      RON     1         Manastur  Imobil vechi      Decomandat  308.0   
4176      RON     4  Andrei Muresanu  Imobil vechi      Decomandat  379.0   
4854      RON     1       Gheorgheni  Imobil vechi             NaN  205.0   

     rooms  space  
175      2   55.0  
177      1   28.0  
1952     1   35.0  
2213     1   28.0  
2694     3   64.0  
2985     2   52.0  
4176     2   60.0  
4854     1   28.0  


After I used the 'RON' category to change its corresponding price into euros I wantto change the name of this category in 'EUR'.

In [67]:
df.loc[df['currency'].str.contains('RON'), 'currency'] = 'EUR'

In [68]:
df.groupby('currency')['currency'].count()

currency
EUR    5002
Name: currency, dtype: int64

I want to see if I have some null values left so above I'm checking this thing.

In [69]:
df.isnull().sum()

currency         0
floor            0
neighborhood     0
new_building     0
partitioning    75
price            0
rooms            0
space            0
dtype: int64

My 'partitioning' feature has 75 null values and, because I don't have many missing values, I can resolve this issue by filling them with the most frequent category of this feature. 

In [70]:
df['partitioning'].value_counts()

Semidecomandat    2512
Decomandat        2348
Nedecomandat        50
Circular             9
Vagon                8
Name: partitioning, dtype: int64

In [71]:
df['partitioning'] = df['partitioning'].fillna(df['partitioning'].mode().iloc[0])

In [72]:
df.isnull().sum()

currency        0
floor           0
neighborhood    0
new_building    0
partitioning    0
price           0
rooms           0
space           0
dtype: int64

In [73]:
df.head()

,currency,floor,neighborhood,new_building,partitioning,price,rooms,space
0,EUR,5,Central,Imobil vechi,Semidecomandat,400.0,2,58.5
1,EUR,8,Manastur,Imobil vechi,Semidecomandat,280.0,2,52.0
2,EUR,3,Zorilor,Imobil nou,Semidecomandat,400.0,3,84.0
3,EUR,1,Gheorgheni,Imobil vechi,Semidecomandat,400.0,3,65.0
4,EUR,1,Zorilor,Imobil nou,Decomandat,420.0,2,55.0


## 2. Feature Engineering

In [74]:
df['price_per_unit_space'] = df['price'] / df['space']
df.head()

,currency,floor,neighborhood,new_building,partitioning,price,rooms,space,price_per_unit_space
0,EUR,5,Central,Imobil vechi,Semidecomandat,400.0,2,58.5,6.837607
1,EUR,8,Manastur,Imobil vechi,Semidecomandat,280.0,2,52.0,5.384615
2,EUR,3,Zorilor,Imobil nou,Semidecomandat,400.0,3,84.0,4.761905
3,EUR,1,Gheorgheni,Imobil vechi,Semidecomandat,400.0,3,65.0,6.153846
4,EUR,1,Zorilor,Imobil nou,Decomandat,420.0,2,55.0,7.636364


In [75]:
number_apartaments_neighborhood = df['neighborhood'].value_counts()
number_apartaments_neighborhood

Gheorgheni             855
Central                765
Marasti                720
Zorilor                559
Manastur               546
Buna Ziua              255
Grigorescu             185
Semicentral            159
Intre Lacuri           155
Andrei Muresanu        125
Europa                 109
Plopilor               107
Ultracentral            95
Iris                    56
P-ta Mihai Viteazul     53
Borhanci                47
Calea Turzii            36
Gruia                   27
Gara                    26
Sopor                   26
Horea                   19
Dambul Rotund           17
Someseni                15
Hasdeu                  14
Bulgaria                10
Aurel Vlaicu             7
Campului                 6
Aeroport                 4
Baciu                    2
Exterior Vest            1
Cordos                   1
Name: neighborhood, dtype: int64

In [76]:
len(number_apartaments_neighborhood[number_apartaments_neighborhood < 10])

6

In [77]:
location_neighborhood_below_10 = number_apartaments_neighborhood[number_apartaments_neighborhood < 10]
location_neighborhood_below_10

Aurel Vlaicu     7
Campului         6
Aeroport         4
Baciu            2
Exterior Vest    1
Cordos           1
Name: neighborhood, dtype: int64

In [78]:
df['neighborhood'] = df['neighborhood'].apply(lambda x: 'Other' if x in location_neighborhood_below_10 else x)
len(df['neighborhood'].unique())

26